In [1]:
from gurobipy import *
import pandas as pd

# #1 Dog Diets

In [2]:
D = {'Uno', 'Dos', 'Tres'}
F = {'Beef', 'Dog Food', 'Bread', 'Bones', 'Chicken'}

C - cost

In [3]:
C = {}
for f in F:
    C[f] = 0
C['Beef'] = 2.5
C['Dog Food'] = 1
C['Bread'] = .8
C['Bones'] = 1.2
C['Chicken'] = 1.6

M - Minimum Food Consumption

In [4]:
M = {}
for d in D:
    M[d] = {}
for m in M:
    for f in F:
        M[m][f] = 0
   

In [5]:
M['Uno']['Bread'] = 0.5
M['Uno']['Bones'] = 1.7
M['Uno']['Chicken'] = 1.9

M['Dos']['Bread'] = 0.3
M['Dos']['Bones'] = .9
M['Dos']['Chicken'] = .1
M['Dos']['Dog Food'] = 1.5

M['Tres']['Bread'] = 0.8
M['Tres']['Bones'] = .6
M['Tres']['Chicken'] = .2
M['Tres']['Dog Food'] = .9
M['Tres']['Beef'] = 1.5

In [6]:
m = Model()
X = m.addVars(D, F, vtype=GRB.CONTINUOUS, name='pounds')

Restricted license - for non-production use only - expires 2023-10-25


In [7]:
cost = LinExpr()
for d in D:
    for f in F:
        cost.add(C[f] * X[d,f])
m.setObjective(cost, GRB.MINIMIZE)

In [8]:
for d in D:
    for f in F:
        pounds = LinExpr()
        pounds.add(X[d,f])
        m.addConstr(pounds >= M[d][f])
        
pounds = LinExpr()
pounds.add(X['Dos', 'Chicken'] + X['Dos', 'Bread'])
m.addConstr(pounds >= 2.5)

pounds = LinExpr()
pounds.add(X['Uno', 'Chicken'] + X['Uno', 'Dog Food'] + X['Uno', 'Bones'])
m.addConstr(pounds >= 2.71)
#m.addConstr(pounds >= 2.5)

pounds = LinExpr()
pounds.add(X['Dos', 'Beef'] + X['Dos', 'Bread'])
m.addConstr(pounds >= 2.61)

<gurobi.Constr *Awaiting Model Update*>

In [9]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 18 rows, 15 columns and 22 nonzeros
Model fingerprint: 0xb9a77336
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 3e+00]
Presolve removed 18 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6638000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.663800000e+01


In [10]:
for d in D:
    for f in F:
        print(f'{d}({f}): {X[d,f].X} pounds')

Uno(Chicken): 1.9 pounds
Uno(Dog Food): 0.0 pounds
Uno(Bread): 0.5 pounds
Uno(Beef): 0.0 pounds
Uno(Bones): 1.7 pounds
Dos(Chicken): 0.1 pounds
Dos(Dog Food): 1.5 pounds
Dos(Bread): 2.61 pounds
Dos(Beef): 0.0 pounds
Dos(Bones): 0.9 pounds
Tres(Chicken): 0.2 pounds
Tres(Dog Food): 0.9 pounds
Tres(Bread): 0.8 pounds
Tres(Beef): 1.5 pounds
Tres(Bones): 0.6 pounds


With Uno's meat consumption constrained at >=2.71 pounds, the cost is minimized at approximately $16.64

Changing Uno's meat consumption to >=2.5 pounds does not change the minimized cost.

# #2 Depots and Neighborhoods

In [11]:
minutes = {
    1 : {1:15, 2:17, 3:27, 4:5, 5:25, 6:22},
    2 : {1:10, 2:12, 3:24 , 4:4 , 5:42 , 6:20 },
    3 : {1:5, 2:6, 3:17 , 4:9 , 5:21 , 6:17 },
    4 : {1:7, 2:6, 3:8 , 4:15 , 5:13 , 6:10 },
    5 : {1:14, 2:12, 3:6 , 4:23 , 5:6 , 6:8 },
    6 : {1:18, 2:17, 3:10 , 4:28 , 5:9 , 6:5 },
    7 : {1:11, 2:10, 3:5 , 4:21 , 5:10 , 6:9 },
    8 : {1:24, 2:22, 3:22 , 4:33 , 5:6 , 6:16 }
}

populations = {
    1: 12,
    2: 8,
    3: 11,
    4: 14,
    5: 22,
    6: 18,
    7: 16,
    8: 20
}


In [12]:
minutes = pd.DataFrame().from_dict(minutes, orient='index')

In [13]:
N = set(minutes.index.tolist())
D = set(minutes.columns.tolist())

In [14]:
T = minutes.to_dict(orient='index')
P = populations

In [15]:
m = Model()

In [16]:
X = m.addVars(D, vtype=GRB.BINARY, name='useDepot?')

In [17]:
popServed = LinExpr()
for d in D:
    for n in N:
        if T[n][d] <= 12:
            popServed.add(X[d] * P[n])
m.setObjective(popServed, GRB.MAXIMIZE)

In [18]:
numDepots = LinExpr()
for d in D:
    numDepots.add(X[d])
m.addConstr(numDepots == 2)

<gurobi.Constr *Awaiting Model Update*>

In [19]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1 rows, 6 columns and 6 nonzeros
Model fingerprint: 0x913cc42f
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 120.0000000
Presolve removed 1 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 147 120 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.470000000000e+02, best bound 1.470000000000e+02, gap 0.0000%


In [20]:
for d in D:
    for n in N:
        if X[d].x == 1:
            print(f'Depot {d} serves Neighborhood {n}')

Depot 2 serves Neighborhood 1
Depot 2 serves Neighborhood 2
Depot 2 serves Neighborhood 3
Depot 2 serves Neighborhood 4
Depot 2 serves Neighborhood 5
Depot 2 serves Neighborhood 6
Depot 2 serves Neighborhood 7
Depot 2 serves Neighborhood 8
Depot 5 serves Neighborhood 1
Depot 5 serves Neighborhood 2
Depot 5 serves Neighborhood 3
Depot 5 serves Neighborhood 4
Depot 5 serves Neighborhood 5
Depot 5 serves Neighborhood 6
Depot 5 serves Neighborhood 7
Depot 5 serves Neighborhood 8


# #3 Big Burger

In [21]:
C = {'Kids', 'Drivers', 'Workers'}
F = {'Burgers', 'Fries', 'Soft Drink', 'Combo Meal'}
D = {
    'Kids' : 300,
    'Drivers' : 240,
    'Workers' : 600,
}


In [22]:
W = {
    'Kids' : 
        {
            'Burgers' : 2.69,
            'Fries' : 1.39,
            'Soft Drink' : 1.09,
            'Combo Meal' : 4.29
        },
    'Drivers' :
        {
            'Burgers' : 2.99,
            'Fries' : .99,
            'Soft Drink' : 1.29,
            'Combo Meal' : 4.89
        },
    'Workers' :
        {
            'Burgers' : 2.59,
            'Fries' : .99,
            'Soft Drink' : 1.19,
            'Combo Meal' : 4.19
        }
}

In [23]:
m = Model()

In [24]:
X = m.addVars(F, vtype=GRB.CONTINUOUS, name='price')
Y = m.addVars(C, F, vtype=GRB.BINARY, name='buy?')

In [25]:
Y

{('Workers', 'Soft Drink'): <gurobi.Var *Awaiting Model Update*>,
 ('Workers', 'Burgers'): <gurobi.Var *Awaiting Model Update*>,
 ('Workers', 'Fries'): <gurobi.Var *Awaiting Model Update*>,
 ('Workers', 'Combo Meal'): <gurobi.Var *Awaiting Model Update*>,
 ('Drivers', 'Soft Drink'): <gurobi.Var *Awaiting Model Update*>,
 ('Drivers', 'Burgers'): <gurobi.Var *Awaiting Model Update*>,
 ('Drivers', 'Fries'): <gurobi.Var *Awaiting Model Update*>,
 ('Drivers', 'Combo Meal'): <gurobi.Var *Awaiting Model Update*>,
 ('Kids', 'Soft Drink'): <gurobi.Var *Awaiting Model Update*>,
 ('Kids', 'Burgers'): <gurobi.Var *Awaiting Model Update*>,
 ('Kids', 'Fries'): <gurobi.Var *Awaiting Model Update*>,
 ('Kids', 'Combo Meal'): <gurobi.Var *Awaiting Model Update*>}

In [26]:
revenue = QuadExpr()
for c in C:
    for f in F:
        revenue.add(X[f] * D[c] * Y[c,f])
m.setObjective(revenue, GRB.MAXIMIZE)

In [27]:
for c in C:
    for f in F:
        isPurchased = QuadExpr()
        isPurchased.add(W[c][f] - (X[f] * Y[c,f]))
        m.addConstr(isPurchased >= 0)

In [28]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 0 rows, 16 columns and 0 nonzeros
Model fingerprint: 0x6c04dc98
Model has 12 quadratic objective terms
Model has 12 quadratic constraints
Variable types: 4 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [1e+00, 5e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 0 rows and 12 columns
Presolve time: 0.01s
Presolved: 9 rows, 13 columns, 18 nonzeros
Presolved model has 6 SOS constraint(s)
Found heuristic solution: objective 5323.8000000
Variable types: 7 continuous, 6 integer (6 binary)

Root relaxation: objective 1.029840e+04, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |

In [29]:
for f in F:
    print(f'Price of {f}: {X[f].x}')

Price of Soft Drink: 1.09
Price of Burgers: 2.5900000000000003
Price of Fries: 0.9900000000000002
Price of Combo Meal: 4.190000000000001


In [32]:
print('Customer Type\tFood Type\tPurchased?(1-yes, 0-no)')
for f in F:
    for c in C:
        print(f'{c}\t\t{f}\t\t\t{Y[c,f].x}')

Customer Type	Food Type	Purchased?(1-yes, 0-no)
Workers		Soft Drink			1.0
Drivers		Soft Drink			1.0
Kids		Soft Drink			1.0
Workers		Burgers			1.0
Drivers		Burgers			1.0
Kids		Burgers			1.0
Workers		Fries			1.0
Drivers		Fries			1.0
Kids		Fries			1.0
Workers		Combo Meal			1.0
Drivers		Combo Meal			1.0
Kids		Combo Meal			1.0
